In [ ]:
# We will not be using simulation here, so this variable should never change
isSimulation = False

# Import Python libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from nptyping import NDArray
from typing import Any, Tuple, List, Optional
import threading

# Import Racecar library
import sys
sys.path.insert(1, "../library")
import racecar_core
import racecar_utils as rc_utils

In [ ]:
# Create Racecar
rc = racecar_core.create_racecar(isSimulation)
rc.go_async()

# Camera

In [ ]:
# Retrieve current image from buffer
image = rc.camera.get_color_image()

# Show image
plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
plt.show()

# LIDAR

In [ ]:
# Access the current LIDAR scan
scan = rc.lidar.get_samples()

# Show lidar data
# Create a square black image with the requested radius
import math

radius = 128
max_range = 400
highlighted_samples=[(0, 100), (90, 100), (180, 100), (270, 100)]

image = np.zeros((2 * radius, 2 * radius, 3), np.uint8, "C")
num_samples: int = len(scan)

# TODO: Draw a green dot at the center of the image to denote the car
# Hint: Use rc_utils.draw_circle
CAR_DOT_RADIUS = 2
rc_utils.draw_circle(
    image,
    (radius, radius),
    (0, 255, 0),
    CAR_DOT_RADIUS,
)
    
# TODO: Draw a red pixel for each non-zero sample less than max_range
for i in range(num_samples):
    if 0 < scan[i] < max_range:
        angle: float = 2 * math.pi * i / num_samples
        length: float = radius * scan[i] / max_range
        r: int = int(radius - length * math.cos(angle))
        c: int = int(radius + length * math.sin(angle))
        image[r][c][2] = 255

# TODO: Draw a light blue dot for each point in highlighted_samples
# Hint: Use rc_utils.draw_circle
HIGHLIGHT_DOT_RADIUS = 2
for (angle, distance) in highlighted_samples:
    if 0 < distance < max_range:
        angle_rad = angle * math.pi / 180
        length: float = radius * distance / max_range
        r: int = int(radius - length * math.cos(angle_rad))
        c: int = int(radius + length * math.sin(angle_rad))
        rc_utils.draw_circle(
            image,
            (r, c),
            (255, 255, 0),
            HIGHLIGHT_DOT_RADIUS
        )

# Show the image with Matplotlib
plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
plt.show()

# IMU

In [ ]:
# Retrieve current IMU values and print to screen
ang_vel = rc.physics.get_angular_velocity()
accel = rc.physics.get_linear_acceleration()
print(f"IMU Values: ang_vel = {ang_vel}, accel = {accel}")

# Dot Matrix

In [ ]:
# Create a new matrix of all zeroes.
my_matrix = rc.display.new_matrix()

# Set the top left pixel to on, then display the matrix on the dot matrix.
my_matrix[0, 0] = 1
rc.display.set_matrix(my_matrix)

In [ ]:
# clear the dot matrix
new_matrix = rc.display.new_matrix()
rc.display.set_matrix(new_matrix)